# Imports and types

In [1]:
import random
import time
import numpy as np
import pandas as pd
import psutil
import os 
import glob

In [2]:
def calcular_aptidao(individuo):
    return sum(individuo)

In [3]:
def mutacao(solucao):
    nova = solucao[:]
    a, b = random.sample(range(len(nova)), 2)
    nova[a], nova[b] = nova[b], nova[a]
    return nova

In [4]:
# gerar_vizinhanca
def gerar_vizinho(solucao):
    return mutacao(solucao)

# Gerar vizinhanças

In [5]:
# M1: Troca de elementos adjacentes
def gerar_vizinhanca_1(solucao):
    vizinhanca = []
    for i in range(len(solucao) - 1):
        vizinho = solucao[:]
        vizinho[i], vizinho[i+1] = vizinho[i+1], vizinho[i]
        vizinhanca.append(vizinho)
    return vizinhanca

In [6]:
# M2: Troca de elementos com distância 2
def gerar_vizinhanca_2(solucao):
    vizinhanca = []
    for i in range(len(solucao) - 2):
        vizinho = solucao[:]
        vizinho[i], vizinho[i+2] = vizinho[i+2], vizinho[i]
        vizinhanca.append(vizinho)
    return vizinhanca

In [7]:
# M3: Todas as trocas possíveis entre pares de posições
def gerar_vizinhanca_3(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n):
        for j in range(i + 1, n):
            vizinho = solucao[:]
            vizinho[i], vizinho[j] = vizinho[j], vizinho[i]
            vizinhanca.append(vizinho)
    return vizinhanca

In [8]:
# M4: Inserção de um elemento em posição anterior
def gerar_vizinhanca_4(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n):
        for j in range(i):
            vizinho = solucao[:]
            elem = vizinho.pop(i)
            vizinho.insert(j, elem)
            vizinhanca.append(vizinho)
    return vizinhanca

In [9]:
# M5: Inserção de um elemento em posição posterior
def gerar_vizinhanca_5(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n):
        for j in range(i+1, n):
            vizinho = solucao[:]
            elem = vizinho.pop(i)
            vizinho.insert(j, elem)
            vizinhanca.append(vizinho)
    return vizinhanca

In [10]:
# M6: Reversão de segmentos de tamanho 2
def gerar_vizinhanca_6(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n-1):
        vizinho = solucao[:]
        vizinho[i:i+2] = reversed(vizinho[i:i+2])
        vizinhanca.append(vizinho)
    return vizinhanca

In [11]:
# M7: Reversão de segmentos de tamanho 3
def gerar_vizinhanca_7(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n-2):
        vizinho = solucao[:]
        vizinho[i:i+3] = reversed(vizinho[i:i+3])
        vizinhanca.append(vizinho)
    return vizinhanca

In [12]:
# M8: 2-opt (reversão de qualquer sublista)
def gerar_vizinhanca_8(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n):
        for j in range(i+2, n):  # precisa pelo menos 2 de diferença
            vizinho = solucao[:]
            vizinho[i:j] = reversed(vizinho[i:j])
            vizinhanca.append(vizinho)
    return vizinhanca

In [13]:
# M9: Swap de blocos de tamanho 2
def gerar_vizinhanca_9(solucao):
    vizinhanca = []
    n = len(solucao)
    for i in range(n-1):
        for j in range(i+2, n-1):
            vizinho = solucao[:]
            vizinho[i:i+2], vizinho[j:j+2] = vizinho[j:j+2], vizinho[i:i+2]
            vizinhanca.append(vizinho)
    return vizinhanca

In [14]:
def selecao_por_torneio(populacao, aptidoes, k=3):
    selecionados = []
    for _ in range(len(populacao)):
        candidatos = random.sample(list(zip(populacao, aptidoes)), k)
        selecionados.append(max(candidatos, key=lambda x: x[1])[0])
    return selecionados

# Croosover

In [15]:
def crossover(pai1, pai2):
    ponto = random.randint(1, len(pai1)-1)
    filho = pai1[:ponto]
    filho.extend([gene for gene in pai2 if gene not in filho])
    return filho

def change_random_paq_positions(solucao):
    nova = solucao[:]
    a, b = random.sample(range(len(nova)), 2)
    nova[a], nova[b] = nova[b], nova[a]
    return nova

def odd_even_crossover(pai1, pai2):
    filho1, filho2 = pai1[:], pai2[:]
    for i in range(len(pai1)):
        if i % 2 == 0:
            filho1[i], filho2[i] = pai2[i], pai1[i]
    return filho1, filho2

def alternate_crossover(pai1, pai2):
    filho = []
    for i in range(len(pai1)):
        if i % 2 == 0:
            filho.append(pai1[i])
        else:
            filho.append(pai2[i])
    return filho

# Mutação

In [16]:
def switch_positions(solucao, pos1, pos2):
    nova = solucao[:]
    nova[pos1], nova[pos2] = nova[pos2], nova[pos1]
    return nova

In [17]:
def random_swap(solucao):
    a, b = random.sample(range(len(solucao)), 2)
    return switch_positions(solucao, a, b)

In [18]:
def gerar_array_replicavel(seed: int, tamanho: int) -> list[int]:
    """Gera um array de inteiros de 0 a tamanho-1 embaralhado, replicável com a mesma semente
        criando populações iniciais diferentes para cada execução e não reiniciando o gerador global de random
    """
    rng = random.Random(seed)   # cria gerador independente
    vetor = list(range(tamanho))
    rng.shuffle(vetor)
    return vetor


# Pareamento

In [19]:
def elite_pairing_vectorized(populacao, aptidoes, num_pares):
    indices_ordenados = np.argsort(aptidoes)[-2*num_pares:]  # pega os índices dos 2*num_pares melhores
    melhores = np.array(populacao)[indices_ordenados]
    pais1 = melhores[:num_pares]
    pais2 = melhores[num_pares:2*num_pares]
    return list(zip(pais1, pais2))

In [20]:
def best_worst_pairing_vectorized(populacao, aptidoes, num_pares):
    indices_ordenados = np.argsort(aptidoes)
    melhores = np.array(populacao)[indices_ordenados[-num_pares:]]
    piores = np.array(populacao)[indices_ordenados[:num_pares]]
    return list(zip(melhores, piores))

In [21]:
def calcular_aptidao_qap(solucao, matriz_fluxo, matriz_distancia):
    n = len(solucao)
    custo = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao[i]][solucao[j]]
    return -custo

In [22]:
def calcular_melhor_vizinho(vizinhanca, matriz_fluxo, matriz_distancia):
    melhor_vizinho = vizinhanca[0]
    melhor_aptidao = calcular_aptidao_qap(melhor_vizinho, matriz_fluxo, matriz_distancia)

    for i in range(1, len(vizinhanca)):
        aptidao = calcular_aptidao_qap(vizinhanca[i], matriz_fluxo, matriz_distancia)
        if aptidao > melhor_aptidao:  
            melhor_vizinho = vizinhanca[i]
            melhor_aptidao = aptidao

    return melhor_vizinho

In [23]:
def VND(solucao, matriz_fluxo, matriz_distancia):
    solucao_atual = solucao[:]
    k = 0
    funcoes_vizinhanca = [gerar_vizinhanca_1, gerar_vizinhanca_2, gerar_vizinhanca_3,
                            gerar_vizinhanca_4, gerar_vizinhanca_5, gerar_vizinhanca_6,
                            gerar_vizinhanca_7, gerar_vizinhanca_8, gerar_vizinhanca_9]
    
    while k < len(funcoes_vizinhanca):
        vizinhos = funcoes_vizinhanca[k](solucao_atual)
        aptidao_atual = calcular_aptidao_qap(solucao_atual, matriz_fluxo, matriz_distancia)
        
        for vizinho in vizinhos:
            aptidao_vizinho = calcular_aptidao_qap(vizinho, matriz_fluxo, matriz_distancia)
            if aptidao_vizinho < aptidao_atual:  
                solucao_atual = vizinho
                k = 0  
                break
        else:
            k += 1  
            
    return solucao_atual

In [24]:
def inicializar_populacao(tamanho_pop, n):
    return [random.sample(range(n), n) for _ in range(tamanho_pop)]
"""
def inicializar_populacao(lambd, solucao_inicial):
    return [mutacao(solucao_inicial) for _ in range(lambd)]
"""

'\ndef inicializar_populacao(lambd, solucao_inicial):\n    return [mutacao(solucao_inicial) for _ in range(lambd)]\n'

# ES_VND

In [25]:
def ES_VND(mu, lambd, tempo_max, taxa_mutacao, taxa_busca_local, iter_sem_melhora_max,
           solucao_inicial, matriz_fluxo, matriz_distancia):

    P = inicializar_populacao(mu + lambd, len(solucao_inicial))
    melhor = None
    melhor_aptidao = float('-inf')  
    sem_melhora = 0

    inicio = time.time()

    while (time.time() - inicio) < tempo_max and sem_melhora < iter_sem_melhora_max:
        aptidoes = [calcular_aptidao_qap(sol, matriz_fluxo, matriz_distancia) for sol in P]

        # Atualiza a melhor solução
        for ind, apt in zip(P, aptidoes):
            if apt > melhor_aptidao:
                melhor_aptidao = apt
                melhor = ind[:]
                sem_melhora = 0
        sem_melhora += 1

        # Selecionar 2 * lambd pais por torneio
        pais = []
        for _ in range(2 * lambd):
            indices = random.sample(range(len(P)), 3)
            candidato = max(indices, key=lambda i: aptidoes[i])
            pais.append(P[candidato])

        # Gerar filhos
        filhos = []
        for i in range(0, 2 * lambd, 2):
            pai1 = pais[i]
            pai2 = pais[i+1]
            filho = crossover(pai1, pai2)
            if random.random() < taxa_mutacao:
                filho = mutacao(filho)
            if random.random() < taxa_busca_local:
                filho = VND(filho, matriz_fluxo, matriz_distancia)
            filhos.append(filho)

        # Selecionar os mu melhores de P
        melhores_indices = sorted(range(len(P)), key=lambda i: aptidoes[i], reverse=True)[:mu]
        melhores = [P[i] for i in melhores_indices]

        # Nova população: mu melhores + lambd filhos
        P = melhores + filhos

    return melhor, melhor_aptidao

# Elistismo

In [26]:
def survivor_eliteism(populacao, aptidoes, mu):
    indices_ordenados = np.argsort(aptidoes)[-mu:]  # pega os índices dos mu melhores
    return [populacao[i] for i in indices_ordenados]

def survivor_tournament_vectorized(populacao, aptidoes, mu, k=3):
    selecionados = []
    n = len(populacao)
    for _ in range(mu):
        candidatos_indices = np.random.choice(n, k, replace=False)
        melhor_indice = candidatos_indices[np.argmax(np.array(aptidoes)[candidatos_indices])]
        selecionados.append(populacao[melhor_indice])
    return selecionados

def best_or_lucky_survivor_vectorized(populacao, aptidoes, mu):
    if random.random() < 0.5:
        return survivor_eliteism(populacao, aptidoes, mu)
    else:
        indices_ordenados = np.argsort(aptidoes)
        piores_indices = indices_ordenados[:len(populacao)-mu]
        sobreviventes = [ind for i, ind in enumerate(populacao) if i not in piores_indices]
        return sobreviventes

# Funções principais

In [27]:
def ler_qap_com_n(caminho: str):
    with open(caminho, "r") as f:
        dados = list(map(int, f.read().split()))
        
    n = dados[0]
    valores = dados[1:]  

    total_esperado = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat = valores[:n * n]
    dist_flat = valores[n * n:]

    flow_df = pd.DataFrame([flow_flat[i * n:(i + 1) * n] for i in range(n)])
    dist_df = pd.DataFrame([dist_flat[i * n:(i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [28]:
if __name__ == "__main__":
    arquivos = glob.glob("*.txt")

    for arquivo in arquivos:
        nome_instancia = os.path.splitext(os.path.basename(arquivo))[0]

        n, flow_df, dist_df = ler_qap_com_n(arquivo)
        flow = flow_df.values.tolist()
        dist = dist_df.values.tolist()
        matriz_fluxo = np.array(flow)
        matriz_distancia = np.array(dist)

        resultados = []  

        for seed in range(42, 52):
            random.seed(seed)

            process = psutil.Process(os.getpid())
            tempo_inicio = time.time()

            solucao_inicial = gerar_array_replicavel(seed=seed, tamanho=n)

            parametros = {
                "mu": random.randint(5, 10),
                "lambd": random.randint(30, 100),
                "tempo_max": random.randint(3, 10) * 60,
                "taxa_mutacao": random.uniform(0.4, 0.7),
                "taxa_busca_local": random.uniform(0.4, 0.7),
                "iter_sem_melhora_max": random.randint(5, 10),
                "solucao_inicial": solucao_inicial,
                "matriz_fluxo": matriz_fluxo,
                "matriz_distancia": matriz_distancia
            }

            melhor_solucao, melhor_valor = ES_VND(**parametros)

            tempo_fim = time.time()

            try:
                memoria_usada = process.memory_info().peak_wset / (1024 * 1024)  
            except AttributeError:
                memoria_usada = process.memory_info().rss / (1024 * 1024)  

            resultados.append({
                "instancia": nome_instancia,
                "seed": seed,
                "melhor_solucao": melhor_solucao,
                "custo": melhor_valor,
                "tempo_execucao_segundos": round(tempo_fim - tempo_inicio, 2),
                "memoria_usada_MB": round(memoria_usada, 2)
            })

            print(f"[{nome_instancia}] Seed {seed} finalizada. Custo: {melhor_valor} | Memória usada: {round(memoria_usada, 2)} MB")

        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_csv(f"resultados_{nome_instancia}.csv", index=False)

        print(f"\nResultados da instância {nome_instancia} salvos em resultados_{nome_instancia}.csv")


[Nug12] Seed 42 finalizada. Custo: -642 | Memória usada: 115.75 MB
[Nug12] Seed 43 finalizada. Custo: -632 | Memória usada: 115.76 MB
[Nug12] Seed 44 finalizada. Custo: -670 | Memória usada: 115.76 MB
[Nug12] Seed 45 finalizada. Custo: -606 | Memória usada: 115.76 MB
[Nug12] Seed 46 finalizada. Custo: -670 | Memória usada: 115.76 MB
[Nug12] Seed 47 finalizada. Custo: -638 | Memória usada: 115.76 MB
[Nug12] Seed 48 finalizada. Custo: -626 | Memória usada: 115.76 MB
[Nug12] Seed 49 finalizada. Custo: -626 | Memória usada: 115.76 MB
[Nug12] Seed 50 finalizada. Custo: -592 | Memória usada: 115.76 MB
[Nug12] Seed 51 finalizada. Custo: -634 | Memória usada: 115.76 MB

Resultados da instância Nug12 salvos em resultados_Nug12.csv
